In [43]:
from glob import glob
import urllib
import earthaccess
from datatree import open_datatree
from PIL import Image
import xarray as xr
from satpy.scene import Scene
import os
import re
from pyresample import create_area_def
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from satpy.writers import get_enhanced_image
from trollimage.xrimage import XRImage
from math import isnan
from PIL import Image
from pyproj import Transformer


## Access and download netCDF files
Data are downloaded from NASA's [EarthData server](https://search.earthdata.nasa.gov/search) to the local directory. This is accomplished using the [earthaccess](https://nsidc.github.io/earthaccess/) package. More information on using EarthData on the cloud is available in the [NASA Earthdata Cloud Cookbook](https://nasa-openscapes.github.io/earthdata-cloud-cookbook/).

1. Need to create and account and login to EarthData using earthaccess.login()
1. Specify which data products to download using NASA's "short names"
   * Products with short names beginning "VNP" are from Suomi-NPP
   * Products with short names beginning "VJ" are from NOAA-20
1. For the Level 1b products, geolocation files accompanying the imagery files are also required
   * Data files have short names containing "02"
   * Geolocation files have short names containing "03"
   
Note that the bounding_box (geographic coordinates of interest) and temporal_limits (time range of interest) should be definted before the download_files() function is called. If a data file contains any data from within the bounding_box, the entire data file will be downloaded.


In [44]:
#Function to query the NASA EarthData archive and download netCDF files into the directories ~/data/level1b/ and ~/data/level2 (if using L2 cloud masks)
def download_files(data_path, bounding_box, temporal_limits):
    '''Download satellite data within a specified geographic and temporal range from NASA EarthData'''
    
    #Requires user to set up an account
    earthaccess.login()  

    #Includes moderate resolution, imagery resolution, and day-night-band data and geolocation files for NOAA-20 and Suomi-NPP
    short_names = ["VNP02MOD", "VNP02IMG", "VNP02DNB", "VNP03MOD", "VNP03IMG", "VNP03DNB",
                      "VJ102MOD", "VJ102IMG", "VJ102DNB", "VJ103MOD", "VJ103IMG", "VJ103DNB"]
      
    #Download data for each file type
    for short_name in short_names:
        if short_name[0:2] == 'VN':
            version = 2
        elif short_name[0:2] == 'VJ':
            version = 2.1
                
        results = earthaccess.search_data(
                    short_name=short_name, 
                    cloud_hosted=True, 
                    bounding_box=bounding_box,
                    temporal=temporal_limits,
                    version=version
        )
        
        #Update to desired directory path
        earthaccess.download(results, data_path)           
            

## Define the projection
All images will be resampled to a common projection. This is required because the tracking algorithms will operate on a pixel-by-pixel basis. This function uses the [create_area_def](https://pyresample.readthedocs.io/en/latest/howtos/geometry_utils.html#areadefinition-creation) function from [pyresample package](https://pyresample.readthedocs.io/en/latest/index.html) (which underlies resampling operations in SatPy, but can be used as a standalone package as well).

Here, data are reprojected to the NSIDC Sea Ice Polar Stereographic North [(EPSG:3413)](https://nsidc.org/data/user-resources/help-center/guide-nsidcs-polar-stereographic-projection) projection. A horizonal resolution of 750 m is chosen. 

I ran into a little trouble implementing this projection which a SatPy engineer helped sort out. Briefly, the "corners" of the projection have to be input from the bounding box. This is likely because SatPy's create_area_def function is getting confused by the polar-centric projection. See this [github discussion](https://github.com/pytroll/tutorial-satpy-half-day/issues/13) for more information about this fix. 

In [45]:

def setup_projection(bounding_box):
    '''Set up a custom area and projection for the images. Input bounding box is (minlon, minlat, maxlon, maxlat)'''

    #Directly defining the extents from the bounding_box doesn't work with this projection. 
    #Instead, use transformer.transform to define corners 
    #See https://github.com/pytroll/tutorial-satpy-half-day/issues/13
    
    #Instead, need to manually determine corners
    transformer = Transformer.from_crs(4326, 3413, always_xy=True) #3413 is the EPSG code of the desired projection

    #Define 'corners'
    (x1, y1) = transformer.transform(bounding_box[0], bounding_box[1])
    (x2, y2) = transformer.transform(bounding_box[0], bounding_box[3])
    (x3, y3) = transformer.transform(bounding_box[2], bounding_box[1])
    (x4, y4) = transformer.transform(bounding_box[2], bounding_box[3])

    x = (x1, x2, x3, x4)
    y = (y1, y2, y3, y4)
    extents = (min(x), min(y), max(x), max(y))
           
    area_id = 'nsidc_polar_north'
    description = 'NSIDC Sea Ice Polar Stereographic North'
    projection = 'EPSG:3413' 
    resolution = 750 #in meters

    my_area = create_area_def(area_id=area_id, description=description, projection=projection, 
                    resolution=resolution, area_extent=extents)    
         
    print(my_area)
    print("Pixel size: x = ", my_area.pixel_size_x, "m, y = ", my_area.pixel_size_y, "m") #horizontal resolution in meters

    return my_area

### Extract the date and time of each set of images from file names
String manipulation function that determines the date and time of each file. The time format is YYYYDDD.HHMM. Returns the list of unique file times (there may multiple files at each file time).

In [46]:
def get_file_times(filenames): 
    '''Generate a list of unique, sorted day and time strings for all files. Result can include times for both VNP and VJ files'''
    
    VNP_pattern = r'\.A(\d{7}).*(\d{4})\.002\.'  #.002 is version
    VJ_pattern = r'\.A(\d{7}).*(\d{4})\.021\.'   #.021 is version 
    
    time_strings = [None] * len(filenames)  # Preallocate with None
    for i, filename in enumerate(filenames):
                
        if 'VN' in filename:
            match = re.search(VNP_pattern, filename)
        elif 'VJ' in filename:
            match = re.search(VJ_pattern, filename)
        
        if match: 
            day = match.group(1)
            hour = match.group(2)
            time_strings[i] = day + '.' + hour
        else:
            print('Cannot determine time for files: ', filenames)
    
    unique_sorted_time_strings = sorted(set(time_strings))
    return(unique_sorted_time_strings) 

## Return the names of all files at a given timestep 
Processed images will be made sequentially by iterating through all *times* for which there is imagery. Multiple files (i.e., imagery-resolution, moderate-resolution, and DNB files) are generated at each image time and need to be grouped together into a single Satpy Scene. Sometimes we also want to group together images that are taken in sequence along the satellite swath. These files are adjacent images taken six minutes apart. This function will check when the previous/next images were taken and group files together if they are on the same swath. 

In [53]:
def get_filenames_at_current_time(data_path, i, filetimes):
    
    '''Generate a list of all filenames at the current timestep. If two sets of files
        are taken six minutes apart, assume they are part of the same swath and group them together, using the earlier time.'''
    
    filetime = filetimes[i]
    filenames = glob(data_path+'*'+filetime+'*.nc') #Files at this timestep
    filetime_dt = datetime.strptime(filetime, "%Y%j.%H%M")

    #Check if the next batch of files is within six minutes and should be combined
    if i < len(filetimes)-1:
        nextfiletime = filetimes[i+1]
        
        # Convert timestamp strings to datetime objects
        nextfiletime_dt = datetime.strptime(nextfiletime, "%Y%j.%H%M")

        # Calculate time difference
        time_difference = nextfiletime_dt - filetime_dt
        combine_files = time_difference <= timedelta(minutes=6) 
        
        if combine_files: 
            print("Combining files at " + filetime[-4:] + ' and ' + nextfiletime[-4:])
            nextfilenames = glob(data_path+'L1b/*'+nextfiletime+'*nc')  
            filenames.extend(nextfilenames)
    
    #Check if this batch of files was already combined with the previous batch of files
    if i > 0:
        prevfiletime = filetimes[i-1]
        prevfiletime_dt = datetime.strptime(prevfiletime, "%Y%j.%H%M")

        # Calculate time difference
        time_difference = filetime_dt - prevfiletime_dt
        already_combined = time_difference <= timedelta(minutes=6)
        if already_combined:
            print("Files at " + filetime[-4:] + ' were aleady combined with ' + prevfiletime[-4:]+ '. Will not reprocess the files')
            filenames = None
            
    #Use version 2 data for VNP - necessary to have this at the end of the function in case files from the next timestep were added
    if filenames is not None:
        filenames = [filename for filename in filenames if '.001.' not in filename]
    
    return filenames

### Determine if the image within the region of interest is from day or night

Determining if the image is daylit is necessary to decide if we want to use Adaptive DNB (for low light conditions) or true and false color imagery (for daylit conditions). 

While the image has a DayNightFlag attribute, this is for the entire large image and can be "both". We want to know if it is day or night within our smaller subregion of interest. From some experimentation, it seems using a maximum solar zenith angle is an effective way to discriminate, with large angles indicating nighttime values (somewhat unintuitive!)

*Note: At the moment I resample the scene here, which seems inefficient as it is later resampled again. However, I think this is unavoidable as the scene must be resampled AFTER the composite is selected, and day or night must first be determined to select which composite to make.* 


In [48]:
def contains_night_pixels(scn, my_area):
    '''Determine if the resampled scene region contains any nighttime pixels''' 
    
    solar_zenith_angles = ['solar_zenith_angle', 'dnb_solar_zenith_angle']
    available = scn.available_dataset_names()
    available_zenith_angles = list(filter(lambda item: item in available, solar_zenith_angles))
    
    # Define a threshold solar zenith angle to distinguish day and night
    max_zenith_angle_threshold  = 88.5 #This is from experimenting
    
    scn.load([available_zenith_angles[0]], generate=False)
    resampled_scene = scn.resample(my_area)
    
    #From experimenting, it seems like the max angle is important in this case. 
    max_zenith_angle = np.nanmax(resampled_scene[available_zenith_angles[0]].values)
       
    #print('Maximum solar angle in resampled scene: ', max_zenith_angle)

    if max_zenith_angle >= max_zenith_angle_threshold or isnan(max_zenith_angle): 
        print('Resampled scene contains some night time pixels')
        return True
    else: 
        print('Resampled scene contains only daytime pixels')
        return False
   

## Download data files, create and save images
Download data from the VIIRS instruments on the S-NPP and NOAA-20 satellites. Make desired composites (i.e., true color images) and save as GeoTIFF files. 

#### Defining a time range and region of interest
The geographic region included in the image can be adjusted by the bounding_box parameter. The time range to make images for can be adjusted by the temporal_limits parameter 

#### Projection
The default projection is the NSIDC Sea Ice Polar Stereographic North projection with horizonal resolution of 750 m. This can potentially be changed within the setup_projection function (defined above). 

#### Image composites
The code determines if the region in bounding_box was daylit/nighttime when the image was taken. If daytime, true color and false color images are made. If nighttime, adaptive DNB images are made. 



In [57]:
#Name the run and directories for data, images 
run = 'test'
data_path = './data_' + run + '/'
image_save_path = './images_' + run + '/'

#Set up time and geographic range
bounding_box=(-155, 72.5, -127, 80) #Central Beaufort Sea
temporal_limits= ('2019-07-06 00:00', '2019-07-06 12:00') #Example time

#Download files (set destination path within download_files)
#download_files(data_path=data_path, bounding_box=bounding_box, temporal_limits=temporal_limits)

#Setup projection for the region of interest for the images
my_area = setup_projection(bounding_box)  

#Create a list of times at which images were taken. These are directly from the file names (the format is YYYYDDD.HHMM)
all_VNP_filenames = glob(data_path+'VNP*.002.*nc') #S-NPP data. 002 indicates Version 2
all_VJ_filenames = glob(data_path+'VJ*.021.*nc') #NOAA-20 data. 021 indicates Version 2.1
filetimes = get_file_times(all_VNP_filenames + all_VJ_filenames) #If you just want images from one instrument, you can use e.g., filetimes = get_file_times(all_VJ_filenames)

print(data_path)

#Make images for each satellite pass
for i, filetime in enumerate(filetimes):
     
    print() # This will print a blank line to distinguish this time's information
    print('Beginning processing images taken at ' + filetime)
    
    #All files at the current time - combine files on same swath but not the same image (i.e., combine pairs of images taken six minutes apart)
    filenames = get_filenames_at_current_time(data_path, i, filetimes)
    
    #filenames will be None if the timestep was part of the same swath as the previous image and was already plotted at previous timestep
    if filenames is None: 
        continue  
    
    # Try to build a Satpy Scene using all files at this timestep. 
    # Skip this timestep if this fails (I have gotten "possibly corrupted file" errors, I think due to downloading errors)
    try:
        scn = Scene(filenames=filenames, reader='viirs_l1b')           
    except Exception as e:
        print(f"An error occurred while processing files at '{filetime}': {e}")
        continue
    
    # Decide what composites to plot
    if contains_night_pixels(scn, my_area):
        channels = ['adaptive_dnb']
    else:
        channels = ['true_color', 'false_color'] 
        
    #Remove any datasets composites that are not available for this scene
    available = scn.available_composite_names() + scn.available_dataset_names()
    channels = list(filter(lambda item: item in available, channels))
      
    #Iterate through the desired composites and make images
    for channel in channels:
         
        #Generate composite and resample to the common grid my_area defined in setup_projection
        try:
 
            #Data will have to be resampled before the composite can actually be generated. 
            #This is because different required bands have different resolution. 
            #generate=False allows us to "load" the ccomposite and then resample
            # See also https://satpy.readthedocs.io/en/latest/faq.html#how-can-i-speed-up-creation-of-composites-that-need-resampling
            scn.load([channel], generate=False)          
            
            #Resample to the common grid defined in my_area
            resampled_scene = scn.resample(my_area)          
            
            #Make and save the image          
            savename = f"{image_save_path}{filetime}_{channel}.tiff"    
            if os.path.exists(savename): #Dont remake this figure
                print(channel+' image at ' + filetime + ' already exists. Will not remake image.')
            
            else:
                print('Saving ' + channel + ' at ' + filetime)# + 'as' + savename)
                
                #Save the image. fill_value=0 excludes the alpha channel and makes an RGB image (techinically it means "set invalid values to black")
                resampled_scene.save_dataset(channel,  writer='geotiff', filename=savename, fill_value=0)

                    
        except Exception as e:
            print(f"An error occurred while processing '{channel}' for '{filetime}': {e}")
            

Rounding shape to (1226, 1162) and resolution from (750.0, 750.0) meters to (749.5606024090914, 749.6601334310984) meters


Area ID: nsidc_polar_north
Description: NSIDC Sea Ice Polar Stereographic North
Projection: {'datum': 'WGS84', 'lat_0': '90', 'lat_ts': '70', 'lon_0': '-45', 'no_defs': 'None', 'proj': 'stere', 'type': 'crs', 'units': 'm', 'x_0': '0', 'y_0': '0'}
Number of columns: 1162
Number of rows: 1226
Area extent: (-1891420.7102, -265821.8454, -1020431.2902, 653261.4782)
Pixel size: x =  749.5606024090915 m, y =  749.6601334310984 m
./data_test/
Beginning processing for images at 2019187.0030
Resampled scene contains only daytime pixels
true_color image at 2019187.0030 already exists. Will not remake image.
false_color image at 2019187.0030 already exists. Will not remake image.
Beginning processing for images at 2019187.0118
Combining files at 0118 and 0124
Resampled scene contains only daytime pixels
Saving true_color at 2019187.0118
Saving false_color at 2019187.0118
Beginning processing for images at 2019187.0124
Files at 0124 were aleady combined with 0118. Will not reprocess the files
Begin